In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `/mnt/064AC6424AC62E6D/git_workspace/flyhopper/scripts/Julia/Project.toml`


In [2]:
using SymPy

In [12]:
# generate manipulator equation
# TODO: Switch to Symbolics.jl

const sp = sympy
sp.init_printing(use_unicode=True)


@syms m0, m1, m2, m3, g, l0, l1, l2, lee, l_c0, l_c1, l_c2, l_cee, I0, I1, I2, I3, alpha0, alpha3
@syms q0()::real t
@syms q1()::real t
@syms q2()::real t
@syms q3()::real t
@syms q0d()::real t
@syms q1d()::real t
@syms q2d()::real t
@syms q3d()::real t
@syms q0dd()::real t
@syms q1dd()::real t
@syms q2dd()::real t
@syms q3dd()::real t

# lee = sqrt((l3 + l4)^2 + l5^2)  # last link is split into l3, l4, and l5

# Assume links do have mass
# Inertia terms are taken w.r.t. CoM of links

x0 = l_c0*sp.cos(q0(t) + alpha0)
y0 = l_c0*sp.sin(q0(t) + alpha0)
x1 = l0*sp.cos(q0(t)) + l_c1*sp.cos(q0(t) + q1(t))
y1 = l0*sp.sin(q0(t)) + l_c1*sp.sin(q0(t) + q1(t))

x2 = l_c2*sp.cos(q2(t))
y2 = l_c2*sp.sin(q2(t))
x3 = l2*sp.cos(q2(t)) + l_cee*sp.cos(q2(t) + q3(t) + alpha3)
y3 = l2*sp.sin(q2(t)) + l_cee*sp.sin(q2(t) + q3(t) + alpha3)

x0d = diff(x0, t)
y0d = diff(y0, t)
x1d = diff(x1, t)
y1d = diff(y1, t)
x2d = diff(x2, t)
y2d = diff(y2, t)
x3d = diff(x3, t)
y3d = diff(y3, t)

U0 = m0*g*y0
U1 = m1*g*y1
U2 = m2*g*y2
U3 = m3*g*y3

v0_squared = x0d^2 + y0d^2
v1_squared = x1d^2 + y1d^2
v2_squared = x2d^2 + y2d^2
v3_squared = x3d^2 + y3d^2
T0 = 0.5*m0*v0_squared + 0.5*I0*q0d(t)^2
T1 = 0.5*m1*v1_squared + 0.5*I1*q1d(t)^2
T2 = 0.5*m2*v2_squared + 0.5*I2*q2d(t)^2 
T3 = 0.5*m3*v3_squared + 0.5*I3*q3d(t)^2

U = U0 + U1 + U2 + U3
T = T0 + T1 + T2 + T3

# Le Lagrangian
L = sp.trigsimp(T - U)
L = L.subs(sp.Derivative(q0(t), t), q0d(t))  # substitute d/dt q2(t) with q2d
L = L.subs(sp.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
L = L.subs(sp.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
L = L.subs(sp.Derivative(q3(t), t), q3d(t))  # substitute d/dt q2(t) with q2d

# Lagrange-Euler Equation
LE0 = diff(diff(L, q0d(t)), t) - diff(L, q0(t))
LE1 = diff(diff(L, q1d(t)), t) - diff(L, q1(t))  
LE2 = diff(diff(L, q2d(t)), t) - diff(L, q2(t))
LE3 = diff(diff(L, q3d(t)), t) - diff(L, q3(t))
LE = [LE0, LE1, LE2, LE3]
# LE = sp.trigsimp(LE)

# subs first derivative
LE = LE.subs(sp.Derivative(q0(t), t), q0d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sp.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
LE = LE.subs(sp.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
LE = LE.subs(sp.Derivative(q3(t), t), q3d(t))  # substitute d/dt q1(t) with q1d
# subs second derivative
LE = LE.subs(sp.Derivative(q0d(t), t), q0dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sp.Derivative(q1d(t), t), q1dd(t))  # substitute d/dt q1d(t) with q1dd
LE = LE.subs(sp.Derivative(q2d(t), t), q2dd(t))  # substitute d/dt q2d(t) with q2dd
LE = LE.subs(sp.Derivative(q3d(t), t), q3dd(t))  # substitute d/dt q1d(t) with q1dd
LE = sp.expand(sp.simplify(LE))

# Generalized mass matrix
# M1 = sp.Poly(LE[1], q1dd(t)).coeffs()
M11 = collect(LE[0], q0dd(t)).coeff(q0dd(t))
M12 = collect(LE[0], q1dd(t)).coeff(q1dd(t))
M13 = collect(LE[0], q2dd(t)).coeff(q2dd(t))
M14 = collect(LE[0], q3dd(t)).coeff(q3dd(t))

M21 = collect(LE[1], q0dd(t)).coeff(q0dd(t))
M22 = collect(LE[1], q1dd(t)).coeff(q1dd(t))
M23 = collect(LE[1], q2dd(t)).coeff(q2dd(t))
M24 = collect(LE[1], q3dd(t)).coeff(q3dd(t))

M31 = collect(LE[2], q0dd(t)).coeff(q0dd(t))
M32 = collect(LE[2], q1dd(t)).coeff(q1dd(t))
M33 = collect(LE[2], q2dd(t)).coeff(q2dd(t))
M34 = collect(LE[2], q3dd(t)).coeff(q3dd(t))

M41 = collect(LE[3], q0dd(t)).coeff(q0dd(t))
M42 = collect(LE[3], q1dd(t)).coeff(q1dd(t))
M43 = collect(LE[3], q2dd(t)).coeff(q2dd(t))
M44 = collect(LE[3], q3dd(t)).coeff(q3dd(t))

M = [M11 M12 M13 M14;
     M21 M22 M23 M24;
     M31 M32 M33 M34;
     M41 M42 M43 M44]

# Gravity Matrix
G = LE
G = G.subs(q0d(t), 0)
G = G.subs(q1d(t), 0)  # must remove q derivative terms manually
G = G.subs(q2d(t), 0)
G = G.subs(q3d(t), 0)
G = G.subs(q0dd(t), 0)
G = G.subs(q1dd(t), 0)
G = G.subs(q2dd(t), 0)
G = G.subs(q3dd(t), 0)

# Coriolis Matrix
# assume anything without qdd minus G is C
C = LE
C = C.subs(q0dd(t), 0)
C = C.subs(q1dd(t), 0)
C = C.subs(q2dd(t), 0)
C = C.subs(q3dd(t), 0)
C = C - G

LE

⎡                                                                             
⎢             1.0⋅I₀⋅q0dd(t) + g⋅l₀⋅m₁⋅cos(q₀(t)) + g⋅l_c0⋅m₀⋅cos(α₀ + q₀(t)) 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢1.0⋅I₂⋅q2dd(t) + g⋅l₂⋅m₃⋅cos(q₂(t)) + g⋅l_c2⋅m₂⋅cos(q₂(t)) + g⋅l_cee⋅m₃⋅cos(α
⎢                                                                             
⎢                                                                             
⎣                                                                           1.

                                   2                                          
+ g⋅l_c1⋅m₁⋅cos(q₀(t) + q₁(t)) + l₀ ⋅m₁⋅q0dd(t) - 2

In [13]:
@show M

M = Sym[1.0*I0 + l0^2*m1 + 2*l0*l_c1*m1*cos(q1(t)) + 1.0*l_c0^2*m0 + l_c1^2*m1 l0*l_c1*m1*cos(q1(t)) + l_c1^2*m1 0 0; 1.0*l0*l_c1*m1*cos(q1(t)) + 1.0*l_c1^2*m1 1.0*I1 + 1.0*l_c1^2*m1 0 0; 0 0 1.0*I2 + l2^2*m3 + 2*l2*l_cee*m3*cos(alpha3 + q3(t)) + 1.0*l_c2^2*m2 + l_cee^2*m3 l2*l_cee*m3*cos(alpha3 + q3(t)) + l_cee^2*m3; 0 0 1.0*l2*l_cee*m3*cos(alpha3 + q3(t)) + 1.0*l_cee^2*m3 1.0*I3 + 1.0*l_cee^2*m3]


4×4 Matrix{Sym}:
 1.0*I0 + l0^2*m1 + 2*l0*l_c1*m1*cos(q1(t)) + 1.0*l_c0^2*m0 + l_c1^2*m1  …                                             0
                              1.0*l0*l_c1*m1*cos(q1(t)) + 1.0*l_c1^2*m1                                                0
                                                                      0     l2*l_cee*m3*cos(alpha3 + q3(t)) + l_cee^2*m3
                                                                      0                          1.0*I3 + 1.0*l_cee^2*m3

In [14]:
@show C

C = SymMatrix(PyObject Matrix([
[                    -2*l0*l_c1*m1*q0d(t)*q1d(t)*sin(q1(t)) - l0*l_c1*m1*q1d(t)**2*sin(q1(t))],
[                                                         1.0*l0*l_c1*m1*q0d(t)**2*sin(q1(t))],
[-2*l2*l_cee*m3*q2d(t)*q3d(t)*sin(alpha3 + q3(t)) - l2*l_cee*m3*q3d(t)**2*sin(alpha3 + q3(t))],
[                                               1.0*l2*l_cee*m3*q2d(t)**2*sin(alpha3 + q3(t))]]))


⎡                                                             2               
⎢      -2⋅l₀⋅l_c1⋅m₁⋅q0d(t)⋅q1d(t)⋅sin(q₁(t)) - l₀⋅l_c1⋅m₁⋅q1d (t)⋅sin(q₁(t)) 
⎢                                                                             
⎢                                          2                                  
⎢                        1.0⋅l₀⋅l_c1⋅m₁⋅q0d (t)⋅sin(q₁(t))                    
⎢                                                                             
⎢                                                              2              
⎢-2⋅l₂⋅l_cee⋅m₃⋅q2d(t)⋅q3d(t)⋅sin(α₃ + q₃(t)) - l₂⋅l_cee⋅m₃⋅q3d (t)⋅sin(α₃ + q
⎢                                                                             
⎢                                        2                                    
⎣                     1.0⋅l₂⋅l_cee⋅m₃⋅q2d (t)⋅sin(α₃ + q₃(t))                 

     ⎤
     ⎥
     ⎥
     ⎥
     ⎥
     ⎥
     ⎥
₃(t))⎥
     ⎥
     ⎥
     ⎦

In [15]:
@show G

G = SymMatrix(PyObject Matrix([
[ g*l0*m1*cos(q0(t)) + g*l_c0*m0*cos(alpha0 + q0(t)) + g*l_c1*m1*cos(q0(t) + q1(t))],
[                                                  1.0*g*l_c1*m1*cos(q0(t) + q1(t))],
[g*l2*m3*cos(q2(t)) + g*l_c2*m2*cos(q2(t)) + g*l_cee*m3*cos(alpha3 + q2(t) + q3(t))],
[                                        1.0*g*l_cee*m3*cos(alpha3 + q2(t) + q3(t))]]))


⎡g⋅l₀⋅m₁⋅cos(q₀(t)) + g⋅l_c0⋅m₀⋅cos(α₀ + q₀(t)) + g⋅l_c1⋅m₁⋅cos(q₀(t) + q₁(t))
⎢                                                                             
⎢                       1.0⋅g⋅l_c1⋅m₁⋅cos(q₀(t) + q₁(t))                      
⎢                                                                             
⎢g⋅l₂⋅m₃⋅cos(q₂(t)) + g⋅l_c2⋅m₂⋅cos(q₂(t)) + g⋅l_cee⋅m₃⋅cos(α₃ + q₂(t) + q₃(t)
⎢                                                                             
⎣                    1.0⋅g⋅l_cee⋅m₃⋅cos(α₃ + q₂(t) + q₃(t))                   

 ⎤
 ⎥
 ⎥
 ⎥
)⎥
 ⎥
 ⎦

In [16]:
T = sympy.simplify(T)
T = T.subs(sympy.Derivative(q0(t), t), q0d(t))  # substitute d/dt q2(t) with q2d
T = T.subs(sympy.Derivative(q1(t), t), q1d(t))  # substitute d/dt q1(t) with q1d
T = T.subs(sympy.Derivative(q2(t), t), q2d(t))  # substitute d/dt q2(t) with q2d
T = T.subs(sympy.Derivative(q3(t), t), q3d(t))  # substitute d/dt q2(t) with q2d
@show T

T = 0.5*I0*q0d(t)^2 + 0.5*I1*q1d(t)^2 + 0.5*I2*q2d(t)^2 + 0.5*I3*q3d(t)^2 + 0.5*l_c0^2*m0*q0d(t)^2 + 0.5*l_c2^2*m2*q2d(t)^2 + 0.5*m1*(l0^2*q0d(t)^2 + 2*l0*l_c1*q0d(t)^2*cos(q1(t)) + 2*l0*l_c1*q0d(t)*q1d(t)*cos(q1(t)) + l_c1^2*q0d(t)^2 + 2*l_c1^2*q0d(t)*q1d(t) + l_c1^2*q1d(t)^2) + 0.5*m3*(l2^2*q2d(t)^2 + 2*l2*l_cee*q2d(t)^2*cos(alpha3 + q3(t)) + 2*l2*l_cee*q2d(t)*q3d(t)*cos(alpha3 + q3(t)) + l_cee^2*q2d(t)^2 + 2*l_cee^2*q2d(t)*q3d(t) + l_cee^2*q3d(t)^2)


          2                2                2                2              2 
0.5⋅I₀⋅q0d (t) + 0.5⋅I₁⋅q1d (t) + 0.5⋅I₂⋅q2d (t) + 0.5⋅I₃⋅q3d (t) + 0.5⋅l_c0 ⋅

      2              2       2             ⎛  2    2                   2      
m₀⋅q0d (t) + 0.5⋅l_c2 ⋅m₂⋅q2d (t) + 0.5⋅m₁⋅⎝l₀ ⋅q0d (t) + 2⋅l₀⋅l_c1⋅q0d (t)⋅co

                                                    2    2            2       
s(q₁(t)) + 2⋅l₀⋅l_c1⋅q0d(t)⋅q1d(t)⋅cos(q₁(t)) + l_c1 ⋅q0d (t) + 2⋅l_c1 ⋅q0d(t)

              2    2   ⎞          ⎛  2    2                    2              
⋅q1d(t) + l_c1 ⋅q1d (t)⎠ + 0.5⋅m₃⋅⎝l₂ ⋅q2d (t) + 2⋅l₂⋅l_cee⋅q2d (t)⋅cos(α₃ + q

                                                        2    2             2  
₃(t)) + 2⋅l₂⋅l_cee⋅q2d(t)⋅q3d(t)⋅cos(α₃ + q₃(t)) + l_cee ⋅q2d (t) + 2⋅l_cee ⋅q

                    2    2   ⎞
2d(t)⋅q3d(t) + l_cee ⋅q3d (t)⎠